## Factors

In [1]:
from zipline.pipeline.data import CNEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage

## Creating a Factor

In [2]:
mean_close_10 = SimpleMovingAverage(
    inputs=[CNEquityPricing.close],
    window_length=10
)

## Adding a Factor to a Pipeline

In [3]:
from zipline.pipeline import Pipeline

In [4]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close], window_length=10)

    return Pipeline(columns={'10_day_mean_close': mean_close_10})

In [5]:
from zipline.research import run_pipeline

In [6]:
result = run_pipeline(make_pipeline(), '2018-04-10', '2018-04-10')
result.head()

10_day_mean_close
2018-04-10 00:00:00+00:00 平安银行(000001)             10.985
                          川化股份(000155)              6.868
                          美的集团(000333)             54.434
                          盐湖股份(000792)             12.969
                          精功科技(002006)              6.371

注意：也可以使用`Pipeline.add`方法将因子添加到现有的`Pipeline`实例中。使用添加看起来像这样：

## Latest

In [7]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close], window_length=10)
    latest_close = CNEquityPricing.close.latest

    return Pipeline(columns={
        '10_day_mean_close': mean_close_10,
        'latest_close_price': latest_close
    })

现在，当我们再次制作并运行我们的管线时，我们的输出数据框中有两列。 一列有每个证券的10日平均收盘价，另一列有最新收盘价。

In [8]:
result = run_pipeline(make_pipeline(), '2018-04-10', '2018-04-10')
result.head()

10_day_mean_close  latest_close_price
2018-04-10 00:00:00+00:00 平安银行(000001)             10.985               10.87
                          川化股份(000155)              6.868                7.03
                          美的集团(000333)             54.434               53.01
                          盐湖股份(000792)             12.969               14.03
                          精功科技(002006)              6.371                6.48

## Default Inputs

一些因子具有不应改变的默认输入。例如，`VWAP`内置因子总是通过`USEquityPricing.close`和`USEquityPricing.volume`进行计算。当总是从相同的`BoundColumns`计算出因子时，我们可以在不指定输入的情况下调用构造函数。

In [9]:
from zipline.pipeline.factors import VWAP
vwap = VWAP(inputs=[CNEquityPricing.close], window_length=10)